In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import random
import math
data_train=pd.read_csv(r"/content/drive/MyDrive/trainMnist.csv")
data_test=pd.read_csv(r"/content/drive/MyDrive/trainMnist.csv")

data_train=np.array(data_train)
data_test=np.array(data_test)

nb_exemple,nb_pixels=data_train.shape
nb_exemple_test,nb_pixels_test=data_test.shape

np.random.shuffle(data_train)
np.random.shuffle(data_test)

data_train=data_train.T
Y_train=data_train[0]
X_train=data_train[1:nb_pixels]
X_train=X_train/255.

data_test=data_test.T
Y_test=data_test[0]
X_test=data_test[1:nb_pixels_test]
X_test=X_test/255.

def init_parametrs():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1,b1,W2,b2,X):
    Z1=W1.dot(X) + b1
    A1=ReLU(Z1)
    Z2=W2.dot(A1) + b2
    A2=softmax(Z2)
    return Z1,A1,Z2,A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y


def ReLU_deriv(Z):
    return Z > 0
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    print("bwckword prop ",Y)
    one_hot_Y = one_hot(Y)
    #erruer
    dZ2 = A2 - one_hot_Y
    #calculer deriver d'erruer hidden layer 2
    dW2 = 1 / 1000 * dZ2.dot(A1.T)
    db2 = 1 / 1000 * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / 1000 * dZ1.dot(X.T)
    db1 = 1 / 1000 * np.sum(dZ1)
    return dW1, db1, dW2, db2

#methode gradient
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

#methode momentieum
def update_parametrs(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha,beta,dV1,dV2,dVB1,dVB2):
    dV1=beta*dV1+(1-beta)*dW1
    W1 = W1 - alpha * dV1

    dVB1=beta*dVB1+(1-beta)*db1
    b1 = b1 - alpha * dVB1

    dV2=beta*dV2+(1-beta)*dW2
    W2 = W2 - alpha * dV2

    dVB2 = beta * dVB2 + (1 - beta) * db2
    b2 = b2 - alpha * dVB2
    return W1, b1, W2, b2,dV1,dV2,dVB1,dVB2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def loss_function(Y_train,Y_pre):
    one_hot_Y = one_hot(Y_train)
    return 0.5 * np.mean((Y_pre - one_hot_Y) ** 2)

In [ ]:
data_train

array([[6, 4, 7, ..., 1, 1, 5],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# **ADAM**

In [ ]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, v1, v2, vb1,vb2,m1,m2,mb1,mb2,v1c,v2c,vb1c,vb2c,m1c,m2c,mb1c,mb2c):
    alpha = 0.001
    beta1 = 0.9
    beta2 = 0.999
    eps = 1e-8

    W1 -= alpha * m1c / (np.sqrt(v1c) + eps)
    W2 -= alpha * m2c / (np.sqrt(v2c) + eps)
    b1 -= alpha * mb1c / (np.sqrt(vb1c) + eps)
    b2 -= alpha * mb2c / (np.sqrt(vb2c) + eps)

    # W1 = W1 - alpha * m1c * 1 / np.sqrt(v1c + eps)
    # W2 = W2 - alpha * m2c * 1 / np.sqrt(v2c + eps)
    # b1 = b1 - alpha * mb1c * 1 / np.sqrt(vb1c + eps)
    # b2 = b2 - alpha * mb2c * 1 / np.sqrt(vb2c + eps)
    # vraiiiiiiiiiiiii
    m1c = m1 / (1 - beta1)
    m2c = m2 / (1 - beta1)
    mb1c = mb1 / (1 - beta1)
    mb2c = mb2 / (1 - beta1)
    # vraiiiiiiiiiiiii
    v1c = v1 / (1 - beta2)
    v2c = v2 / (1 - beta2)
    vb1c = vb1 / (1 - beta2)
    vb2c = vb2 / (1 - beta2)
    # vraiiiiiiii


    # m1 = beta1 * m1 + (1 - beta1) * dW1
    # m2 = beta1 * m2 + (1 - beta1) * dW2
    # mb1 = beta1 * mb1 + (1 - beta1) * db1
    # mb2 = beta1 * mb2 + (1 - beta1) * db2
    #
    # v1 = beta2 * v1 + (1 - beta2) * (dW1**2)
    # v2 = beta2 * v2 + (1 - beta2) * (dW2**2)
    # vb1 = beta2 * vb1 + (1 - beta2) * (db1**2)
    # vb2 = beta2 * vb2 + (1 - beta2) * (db2**2)
    m1 = beta1 * m1 + (1 - beta1) * dW1
    mb1 = beta1 * mb1 + (1 - beta1) * db1
    m2 = beta1 * m2 + (1 - beta1) * dW2
    mb2 = beta1* mb2 + (1 - beta1) * db2
    # vraiiiiiiii
    v1 = beta2 * v1 + (1 - beta2) * (dW1**2)
    vb1 = beta2 * vb1 + (1 - beta2) *( db1**2)
    v2 = beta2 * v2 + (1 - beta2) * (dW2**2)
    vb2 = beta2* vb2 + (1 - beta2) * (db2**2)

    return W1, b1, W2, b2, dW1, db1, dW2, db2, v1, v2, vb1,vb2,m1,m2,mb1,mb2,v1c,v2c,vb1c,vb2c,m1c,m2c,mb1c,mb2c


def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_parametrs()
    v1 = np.zeros(W1.shape)
    vb1 = np.zeros(b1.shape)
    v2 = np.zeros(W2.shape)
    vb2 = np.zeros(b2.shape)

    v1c = np.zeros(W1.shape)
    vb1c = np.zeros(b1.shape)
    v2c = np.zeros(W2.shape)
    vb2c = np.zeros(b2.shape)

    m1c = np.zeros(W1.shape)
    mb1c = np.zeros(b1.shape)
    m2c = np.zeros(W2.shape)
    mb2c = np.zeros(b2.shape)

    m1 = np.zeros(W1.shape)
    mb1 = np.zeros(b1.shape)
    m2 = np.zeros(W2.shape)
    mb2 = np.zeros(b2.shape)


    beta=0.9
    W1, b1, W2, b2 = init_parametrs()
    dV1= np.zeros(W1.shape)
    dVB1=np.zeros(b1.shape)
    dV2=np.zeros(W2.shape)
    dVB2=np.zeros(b2.shape)
    batch_Y=Y[0:1000]
    for i in range(iterations):
       # np.random.shuffle(X)
       # np.random.shuffle(Y)
        for p in range(0,nb_exemple,1000):
            batch_X=X[:,p:p+1000]
            batch_Y=Y[p:p+1000]
            Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
            dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
            W1, b1, W2, b2, dW1, db1, dW2, db2, v1, v2, vb1,vb2,m1,m2,mb1,mb2,v1c,v2c,vb1c,vb2c,m1c,m2c, mb1c,mb2c = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, v1, v2,vb1,vb2,m1,m2,mb1,mb2,v1c,v2c,vb1c,vb2c,m1c,m2c,mb1c,mb2c)
            if i % 10 == 0:

              print("Iteration: ", i)
              predictions = get_predictions(A2)
              print(get_accuracy(predictions, Y))

W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10,2)

bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.05383333333333333
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.05383333333333333
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.05383333333333333
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.05564285714285714
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.05904761904761905
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.063
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.06728571428571428
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.07273809523809524
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.07671428571428572
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.08102380952380953
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.0855
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.08952380952380952
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.09411904761904762
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.09802380952380953
bwckword prop  [9 5 2 ... 9 1 8]
Iteration:  0
0.10102380952380953
bwckword prop  [9 5 2 

TypeError: ignored

# **ada grad**

In [ ]:

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, v1, v2, vb1, vb2):
    eps=0.00000001

    v1=v1+dW1**2
    v2=v2+dW2**2
    vb1=vb1+db1**2
    vb2=vb2+db2**2

    W1 = W1 - alpha * dW1 *1/ np.sqrt(v1 + eps)
    W2 = W2 - alpha * dW2 *1/ np.sqrt(v2 + eps)
    b1 = b1 - alpha * db1 *1/ np.sqrt(vb1 + eps)
    b2 = b2 - alpha * db2 *1/np.sqrt(vb2 + eps)

    # beta = 0.9
    # W1 = W1 - alpha * v1
    # b1 = b1 - alpha * vb1
    # W2 = W2 - alpha * v2
    # b2 = b2 - alpha * vb2
    # v1 = beta * v1 + (1 - beta) * dW1
    # vb1 = beta * vb1 + (1 - beta) * db1
    # v2 = beta * v2 + (1 - beta) * dW2
    # vb2 = beta * vb2 + (1 - beta) * db2

    return W1, b1, W2, b2, v1, v2, vb1, vb2

def gradient_descent(X, Y, alpha, iterations):
    beta=0.9
    W1, b1, W2, b2 = init_parametrs()
    dV1= np.zeros(W1.shape)
    dVB1=np.zeros(b1.shape)
    dV2=np.zeros(W2.shape)
    dVB2=np.zeros(b2.shape)
    batch_Y=Y[0:1000]
    w11 = np.zeros(W1.shape)
    b11 = np.zeros(b1.shape)
    w22 = np.zeros(W2.shape)
    b22 = np.zeros(b2.shape)
    for i in range(iterations):
       # np.random.shuffle(X)
       # np.random.shuffle(Y)
        for p in range(0,nb_exemple,1000):
            batch_X=X[:,p:p+1000]
            batch_Y=Y[p:p+1000]
            Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
            dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
            W1, b1, W2, b2, w11, w22, b11, b22 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, w11, w22, b11,b22)
            if i % 10 == 0:
                print("Iteration: ", i)
                predictions = get_predictions(A2)
                print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10,10)

Iteration:  0
0.05604761904761905
Iteration:  0
0.14642857142857144
Iteration:  0
0.1789047619047619
Iteration:  0
0.25997619047619047
Iteration:  0
0.19138095238095237
Iteration:  0
0.2235
Iteration:  0
0.2917619047619048
Iteration:  0
0.3028571428571429
Iteration:  0
0.3259047619047619
Iteration:  0
0.3478809523809524
Iteration:  0
0.3645
Iteration:  0
0.37242857142857144
Iteration:  0
0.37907142857142856
Iteration:  0
0.38621428571428573
Iteration:  0
0.3916428571428571
Iteration:  0
0.3987142857142857
Iteration:  0
0.4037142857142857
Iteration:  0
0.41383333333333333
Iteration:  0
0.42192857142857143
Iteration:  0
0.4205238095238095
Iteration:  0
0.4111904761904762
Iteration:  0
0.42545238095238097
Iteration:  0
0.4519285714285714
Iteration:  0
0.45054761904761903
Iteration:  0
0.46676190476190477
Iteration:  0
0.4575714285714286
Iteration:  0
0.4707857142857143
Iteration:  0
0.4531428571428571
Iteration:  0
0.4635
Iteration:  0
0.453
Iteration:  0
0.47559523809523807
Iteration:  0

# RMS **PROP**

In [ ]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, v1, v2, vb1, vb2):
    eps=0.00000001
    beta = 0.99
    v1=beta*v1+(1-beta)*dW1**2
    v2=beta*v2+(1-beta)*dW2**2
    vb1=beta*vb1+(1-beta)*db1**2
    vb2=beta*vb2+(1-beta)*db2**2
    W1 = W1 - alpha * dW1 *1/ np.sqrt(v1 + eps)
    W2 = W2 - alpha * dW2 *1/ np.sqrt(v2 + eps)
    b1 = b1 - alpha * db1 *1/ np.sqrt(vb1 + eps)
    b2 = b2 - alpha * db2 *1/np.sqrt(vb2 + eps)

    return W1, b1, W2, b2, v1, v2, vb1, vb2


def gradient_descent(X, Y, alpha, iterations):
    beta=0.9
    W1, b1, W2, b2 = init_parametrs()
    dV1= np.zeros(W1.shape)
    dVB1=np.zeros(b1.shape)
    dV2=np.zeros(W2.shape)
    dVB2=np.zeros(b2.shape)
    batch_Y=Y[0:1000]
    w11 = np.zeros(W1.shape)
    b11 = np.zeros(b1.shape)
    w22 = np.zeros(W2.shape)
    b22 = np.zeros(b2.shape)
    for i in range(iterations):
          # np.random.shuffle(X)
          # np.random.shuffle(Y)
            for p in range(0,nb_exemple,1000):
                batch_X=X[:,p:p+1000]
                batch_Y=Y[p:p+1000]
                Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
                dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
                W1, b1, W2, b2, w11, w22, b11, b22 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, w11, w22, b11, b22)
                if i % 10 == 0:
                    print("Iteration: ", i)
                    predictions = get_predictions(A2)
                    print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2


W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

# Adadelta **texte en gras**

In [ ]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, v1, v2, vb1, vb2,D1,D2,Db1,Db2):
    w1_old = W1.copy()
    w2_old = W2.copy()
    b1_old = b1.copy()
    b2_old = b2.copy()

    eps=0.00000001
    beta = 0.9

    v1 = beta * v1 + (1 - beta) * (dW1 ** 2)
    v2 = beta * v2 + (1 - beta) * (dW2 ** 2)
    vb1 = beta * vb1 + (1 - beta) * (db1 ** 2)
    vb2 = beta * vb2 + (1 - beta) * (db2 ** 2)



    W1 -= (np.sqrt(D1) + eps) * dW1 * 1 / np.sqrt(v1 + eps)
    W2 -= (np.sqrt(D2) + eps) * dW2 * 1 / np.sqrt(v2 + eps)
    b1 -= (np.sqrt(Db1) + eps) * db1 * 1 / np.sqrt(vb1 + eps)
    b2 -= (np.sqrt(Db2) + eps) * db2 * 1 / np.sqrt(vb2 + eps)

    D1 = beta * D1 + (1 - beta) * ((w1_old - W1) ** 2)
    D2 = beta * D2 + (1 - beta) * ((w2_old - W2) ** 2)
    Db1 = beta * Db1 + (1 - beta) * ((b1_old - b1) ** 2)
    Db2 = beta * Db2 + (1 - beta) * ((b2_old - b2) ** 2)


    return W1, b1, W2, b2, v1, v2, vb1, vb2 ,D1 ,D2 ,Db1 ,Db2

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_parametrs()
    w11 = np.zeros(W1.shape)
    b11 = np.zeros(b1.shape)
    w22 = np.zeros(W2.shape)
    b22 = np.zeros(b2.shape)
    D1 = np.zeros(W1.shape)
    Db1 = np.zeros(b1.shape)
    D2 = np.zeros(W2.shape)
    Db2 = np.zeros(b2.shape)
    for i in range(iterations):
          # np.random.shuffle(X)
          # np.random.shuffle(Y)
            for p in range(0,nb_exemple,1000):
                batch_X=X[:,p:p+1000]
                batch_Y=Y[p:p+1000]
                Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
                dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
                W1, b1, W2, b2, w11, w22, b11, b22,D1,D2,Db1,Db2= update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, w11, w22, b11,b22,D1,D2,Db1,Db2)
                if i % 10 == 0:
                    print("Iteration: ", i)
                    predictions = get_predictions(A2)
                    print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2


W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 100)